# Organizing the Data

## Set up Label Folders

In [1]:
from pathlib import Path

import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
master_folder = Path("EMG_data_for_gestures-master")
output_master_folder = master_folder / "Processed_Data"

In [3]:
labels = {
    1: "1_hand_at_rest",
    2: "2_hand_clenched_in_a_fist",
    3: "3_wrist_flexion",
    4: "4_wrist_extension",
    5: "5_radial_deviations",
    6: "6_ulnar_deviations",
    7: "7_extended_palm"
}

In [4]:
# Create output folders
output_master_folder.mkdir(exist_ok=True)
for label_name in labels.values():
    (output_master_folder / label_name).mkdir(exist_ok=True)

In [5]:
for i in range(1, 2): # Change 2 to 37 when ready
    f_num = i
    subfolder = os.path.join(master_folder, str(f_num))
    if f_num < 10:
        subfolder = os.path.join(master_folder, "0" + str(f_num))

    for filename in os.listdir(subfolder):
        if filename.endswith(".txt"):
            filepath = os.path.join(subfolder, filename)
            # Step 1: Read the file
            df = pd.read_csv(filepath, delim_whitespace=True)

            # Step 2: Remove rows with class label 0
            df = df[df["class"] != 0]

            # Step 3: Group data by contiguous blocks of the same label
            current_label = None
            segment_data = []
            segment_count = {label: 0 for label in labels.values()}

            for _, row in df.iterrows():
                if row["class"] != current_label:
                    # Save the previous segment if it exists
                    if segment_data:
                        segment_df = pd.DataFrame(segment_data, columns=df.columns)
                        label_name = labels[int(current_label)]
                        segment_count[label_name] += 1
                        segment_path = output_master_folder / label_name / f"segment_{segment_count[label_name]}.csv"
                        segment_df.to_csv(segment_path, index=False)
                        segment_data = []
                    current_label = row["class"]

                segment_data.append(row)

            # Save the final segment
            if segment_data:
                segment_df = pd.DataFrame(segment_data, columns=df.columns)
                label_name = labels[int(current_label) - 1]
                segment_count[label_name] += 1
                segment_path = output_master_folder / label_name / f"segment_{segment_count[label_name]}.csv"
                segment_df.to_csv(segment_path, index=False)